In [1]:
from __future__ import print_function, division
from builtins import range

import numpy as np

In [2]:
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
        
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
    
    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]
        
    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions
    
    def move(self, action):
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1
        return self.rewards.get((self.i, self.j), 0)
    
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        assert(self.current_state() in self.all_states())
    
    def game_over(self):
        return (self.i, self.j) not in self.actions
    
    def all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())
    


In [3]:
def standard_grid():
    g = Grid(3, 4, (2,0))
    rewards = {(0,3):1, (1,3):-1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
              }
    g.set(rewards, actions)
    return g

def negative_grid(step_cost=-0.1):
    g = standard_grid()
    g.rewards.update({
        (0, 0): step_cost,
        (0, 1): step_cost,
        (0, 2): step_cost,
        (1, 0): step_cost,
        (1, 2): step_cost,
        (2, 0): step_cost,
        (2, 1): step_cost,
        (2, 2): step_cost,
        (2, 3): step_cost,
              })
    return g
    
    

In [4]:
SMALL_ENOUGH = 1e-3

def print_values(V, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            v = V.get((i,j), 0)
            if v>=0:
                print(" %.2f|"%v, end='')
            else:
                print("%.2f|"%v, end='')
        print()

def print_policy(P, g):
    for i in range(g.width):
        print("---------------------------")
        for j in range(g.height):
            a = P.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [5]:
grid = standard_grid()
states = grid.all_states()
print(states)

{(0, 1), (1, 2), (0, 0), (1, 3), (2, 1), (2, 0), (2, 3), (2, 2), (1, 0), (0, 2), (0, 3)}


In [6]:
V = dict()
for s in states:
    V[s] = 0
gamma = 1.0
while True:
    biggest_change = 0
    for s in states:
        old_v = V[s]
        if s in grid.actions:
            new_v = 0
            p_a = 1.0/len(grid.actions[s])
            for a in grid.actions[s]:
                grid.set_state(s)
                r = grid.move(a)
                new_v += p_a*(r+gamma*V[grid.current_state()])
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
            
    if biggest_change < SMALL_ENOUGH:
        break
print("values for uniformly random actions: ")
print_values(V, grid)

values for uniformly random actions: 
---------------------------
-0.03| 0.09| 0.22| 0.00|
---------------------------
-0.16| 0.00|-0.44| 0.00|
---------------------------
-0.29|-0.41|-0.54|-0.77|


In [7]:
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'R',
    (2, 1): 'R',
    (2, 2): 'R',
    (2, 3): 'U',
  }
print_policy(policy, grid)

---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |


In [8]:
V = {}
for s in states:
    V[s] = 0

gamma = 0.9

while True:
    biggest_change = 0
    for s in states:
        old_v = V[s]
        
        if s in policy:
            a = policy[s]
            grid.set_state(s)
            r = grid.move(a)
            V[s] = r + gamma*V[grid.current_state()]
            biggest_change = max(biggest_change, np.abs(old_v-V[s]))
    if biggest_change < SMALL_ENOUGH:
        break
print('values for fixed policy')
print_values(V, grid)

values for fixed policy
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00|-1.00| 0.00|
---------------------------
 0.66|-0.81|-0.90|-1.00|


In [9]:
# Policy Improvement (optimal policy identification)

GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U','D','L','R')

grid = negative_grid()

print('rewards: ')
print_values(grid.rewards, grid)

rewards: 
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [10]:
policy = dict()
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
print('initial policy: ')
print_policy(policy, grid)

initial policy: 
---------------------------
  D  |  L  |  D  |     |
---------------------------
  D  |     |  D  |     |
---------------------------
  D  |  D  |  R  |  R  |


In [11]:
V = dict()
states = grid.all_states()
for s in states:
    V[s] = 0
while True:
    # policy evaluation
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]
            if s in policy:
                a = policy[s]
                grid.set_state(s)
                r = grid.move(a)
                V[s] = r + GAMMA*V[grid.current_state()]
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        if biggest_change < SMALL_ENOUGH:
            break
    is_policy_converged = True
    
    # policy improvement
    for s in states:
        if s in policy:
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                v = r+GAMMA*V[grid.current_state()]
                if v>best_value:
                    best_value = v
                    new_a = a
            policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False
    if is_policy_converged:
        break
print("values: ")
print_values(V, grid)
print()
print('policy: ')
print_policy(policy, grid)

values: 
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|

policy: 
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


In [12]:
##### windy grid world

grid = negative_grid(step_cost=-0.1) # try step_cost = -1.0 , observe bot tries to complete as fast as possible
print("rewards: ")
print_values(grid.rewards, grid)

rewards: 
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [13]:
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
print('initial policy: ')
print_policy(policy, grid)

initial policy: 
---------------------------
  D  |  R  |  L  |     |
---------------------------
  U  |     |  L  |     |
---------------------------
  U  |  R  |  U  |  U  |


In [14]:
V = dict()
states = grid.all_states()
for s in states:
    V[s] = 0

In [15]:
while True:
    #policy evaluation in windy grid world
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]
            new_v = 0
            if s in policy:
                for a in ALL_POSSIBLE_ACTIONS:
                    if a == policy[s]:
                        p = 0.5
                    else:
                        p = 0.5/3
                    grid.set_state(s)
                    r = grid.move(a)
                    new_v += p*(r+GAMMA*V[grid.current_state()])
                V[s] = new_v
                biggest_change = max(biggest_change, np.abs(old_v-V[s]))
        if biggest_change < SMALL_ENOUGH:
            break
    # policy improvement step
    is_policy_converged = True
    for s in states:
        if s in policy:
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            
            for a in ALL_POSSIBLE_ACTIONS:
                v = 0
                for a2 in ALL_POSSIBLE_ACTIONS:
                    if a == a2:
                        p = 0.5
                    else:
                        p = 0.5/3
                    grid.set_state(s)
                    r = grid.move(a2)
                    v += p*(r+GAMMA*V[grid.current_state()])
                if v>best_value:
                    best_value = v
                    new_a = a
            policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False
    if is_policy_converged:
        break
    

In [16]:
print("values:")
print_values(V, grid)
print("policy:")
print_policy(policy, grid)

values:
---------------------------
-0.08| 0.20| 0.55| 0.00|
---------------------------
-0.28| 0.00|-0.06| 0.00|
---------------------------
-0.42|-0.44|-0.33|-0.57|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |


In [17]:
# value iteration method..

In [18]:
grid = negative_grid()
print('rewards: ')
print_values(grid.rewards, grid)

rewards: 
---------------------------
-0.10|-0.10|-0.10| 1.00|
---------------------------
-0.10| 0.00|-0.10|-1.00|
---------------------------
-0.10|-0.10|-0.10|-0.10|


In [19]:
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
print('initial policy')
print_policy(policy, grid)

initial policy
---------------------------
  U  |  U  |  L  |     |
---------------------------
  U  |     |  D  |     |
---------------------------
  L  |  L  |  D  |  R  |


In [20]:
V = dict()
states = grid.all_states()
for s in states:
    V[s] = 0

In [22]:
while True:
    biggest_change = 0
    for s in states:
        old_v = V[s]
        if s in policy:
            new_v = float('-inf')
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                v = r+GAMMA*V[grid.current_state()]
                if v>new_v:
                    new_v = v
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
    if biggest_change < SMALL_ENOUGH:
        break
for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        v = r + GAMMA*V[grid.current_state()]
        if v>best_value:
            best_value = v
            best_a = a
    policy[s] = best_a
print('values:')
print_values(V, grid)
print('policy: ')
print_policy(policy, grid)
    
    

values:
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00| 0.80| 0.00|
---------------------------
 0.31| 0.46| 0.62| 0.46|
policy: 
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
